# Bash tasks

This page will cover the peculiarities of using bash to describe tasks in airflow.

## DAG

In [50]:
%%writefile bash_tasks/bash_tasks.py

from datetime import datetime, timedelta
from textwrap import dedent

from airflow import DAG

from airflow.operators.bash import BashOperator
with DAG(
    "bash_tasks",
    default_args={
        "depends_on_past": False,
    },
    description="A simple tutorial DAG",
    schedule=timedelta(days=1),
    start_date=datetime(2021, 1, 1),
    catchup=False,
    tags=["example"],
) as dag:

    print_bash_date = BashOperator(
        task_id="print_date",
        bash_command="""
        for i in {0..3}
        do
            echo "current date: $(date)"
        done
        """
    )

    run_external_script = BashOperator(
        task_id="external_script",
        bash_command="""
        for i in {0..3}
        do
            echo "current date: $(date)"
        done
        """
    )

Overwriting bash_tasks/bash_tasks.py


## Container

In [51]:
%%bash
docker run -d --rm\
    --name bash_tasks\
    -p 8080:8080\
    -v ./bash_tasks:/root/airflow/dags\
    airflow_tests &> /dev/null

docker exec bash_tasks airflow dags list

dag_id     | filepath      | owner   | paused
===========+===============+=========+=======
bash_tasks | bash_tasks.py | airflow | True  
                                             


In [52]:
!docker exec bash_tasks airflow db migrate

DB: sqlite:////root/airflow/airflow.db
Performing upgrade to the metadata database sqlite:////root/airflow/airflow.db
[2023-09-25T11:23:17.657+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-09-25T11:23:17.658+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-09-25T11:23:17.659+0000] {db.py:1622} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Database migrating done!


In [46]:
!docker stop bash_tasks

bash_tasks


## `print_bash_date` 

In [53]:
!docker exec bash_tasks airflow tasks test bash_tasks print_date 2015-06-01
#| grep " - current date"

[2023-09-25T11:23:20.435+0000] {dagbag.py:539} INFO - Filling up the DagBag from /root/airflow/dags
[2023-09-25T11:23:20.558+0000] {taskinstance.py:1157} INFO - Dependencies all met for dep_context=non-requeueable deps ti=<TaskInstance: bash_tasks.print_date __airflow_temporary_run_2023-09-25T11:21:25.160046+00:00__ [None]>
[2023-09-25T11:23:20.561+0000] {taskinstance.py:1157} INFO - Dependencies all met for dep_context=requeueable deps ti=<TaskInstance: bash_tasks.print_date __airflow_temporary_run_2023-09-25T11:21:25.160046+00:00__ [None]>
[2023-09-25T11:23:20.562+0000] {taskinstance.py:1359} INFO - Starting attempt 1 of 2
[2023-09-25T11:23:20.562+0000] {taskinstance.py:1428} WARNING - cannot record queued_duration for task print_date because previous state change time has not been saved
[2023-09-25T11:23:20.562+0000] {taskinstance.py:1380} INFO - Executing <Task(BashOperator): print_date> on 2015-06-01 00:00:00+00:00
[2023-09-25T11:23:20.581+0000] {taskinstance.py:1660} INFO - Expor